In [1]:
import pandas as pd
import numpy as np
import os
import sys
import gc
from sklearn.manifold import TSNE
from importlib import reload
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
import scipy 

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import utils
reload(utils)

<module 'utils' from '/Users/milk/Desktop/git_repositories/READMISSIONS/washtenaw_case_study/notebooks/utils.py'>

In [2]:
matplotlib.rcParams['figure.dpi'] = 200
plt.style.use('seaborn-deep')

In [7]:
DATA_PATH = "../cleaned_data/clean_data.csv"
data_df = pd.read_csv(DATA_PATH)
data_df['TRACT'] = data_df['TRACT'].astype(str)
print(data_df.shape)
data_df.head()

(5013, 3881)


,Abseiling_label,Adaptation_label,Adventure_label,Aerial_photography_label,Amusement_park_label,Amusement_ride_label,Ancient_rome_label,Antique_label,Apartment_label,Arcade_label,...,blue_eig_991,blue_eig_992,blue_eig_993,blue_eig_994,blue_eig_995,blue_eig_996,blue_eig_997,blue_eig_998,blue_eig_999,blue_eig_1000
0,0,0,0,0,0,0,0,0,0,0,...,2.869129,2.537908,2.063795,2.020026,1.771241,1.563943,0.896316,0.494682,0.211528,0.044257
1,0,0,0,0,0,0,0,0,0,0,...,3.470981,3.009629,2.361901,1.923912,1.751543,1.265105,1.083160,0.749636,0.654097,0.226061
2,0,0,0,0,0,0,0,0,0,0,...,3.291750,2.898913,2.588453,2.412453,1.951029,1.512802,1.440446,0.998019,0.539556,0.048014
3,0,0,0,0,0,0,0,0,0,0,...,3.410401,3.115535,2.783957,2.237531,1.509916,1.329670,1.128742,0.854100,0.407354,0.213916
4,0,0,0,0,0,0,0,0,0,0,...,3.243908,3.191757,2.773401,2.312393,1.647995,1.508362,0.994718,0.816073,0.257799,0.050827


In [8]:
CITIES_PATH = "../demographic_data/Ann_Arbor_500_Cities.csv"
cities_df = pd.read_csv(CITIES_PATH)
print(cities_df.shape)
cities_df = cities_df[cities_df['PlaceName'] == 'Ann Arbor']
print(f"Ann Arbor tracts: {cities_df.shape}")
cities_df['TRACT'] = cities_df['TractFIPS'].astype(str).str[-6:]
cities_df.head()

(27210, 63)
Ann Arbor tracts: (33, 63)


,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,population_count,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,TRACT
13525,MI,Ann Arbor,2603000,26161400100,2603000-26161400100,1755,12.8,"(10.2, 16.6)",11.3,"(10.6, 11.9)",...,7.9,"( 6.8, 9.1)",32.9,"(30.1, 35.4)",1.5,"( 1.4, 1.7)",14.2,"(10.1, 19.2)","(42.28010295770, -83.7462540302)",400100
13526,MI,Ann Arbor,2603000,26161400200,2603000-26161400200,7098,14.1,"(11.8, 16.2)",4.3,"( 3.8, 4.7)",...,6.1,"( 5.2, 6.7)",31.3,"(28.7, 33.3)",0.4,"( 0.4, 0.5)",11.9,"( 8.7, 14.6)","(42.28154505960, -83.7324117791)",400200
13527,MI,Ann Arbor,2603000,26161400300,2603000-26161400300,6222,19.1,"(14.6, 26.1)",6.5,"( 5.6, 7.3)",...,8.8,"( 7.0, 10.9)",33.9,"(29.6, 37.2)",0.9,"( 0.7, 1.0)",20.3,"(15.7, 25.9)","(42.27454257450, -83.7303535678)",400300
13528,MI,Ann Arbor,2603000,26161400400,2603000-26161400400,3045,7.4,"( 6.2, 9.0)",19.2,"(18.2, 20.3)",...,8.4,"( 7.5, 9.3)",29.0,"(27.2, 30.9)",1.9,"( 1.7, 2.1)",7.3,"( 5.0, 9.9)","(42.26482251930, -83.7310920606)",400400
13529,MI,Ann Arbor,2603000,26161400500,2603000-26161400500,6088,18.8,"(13.1, 25.7)",7.3,"( 6.5, 8.2)",...,9.2,"( 7.1, 11.4)",33.5,"(29.3, 36.8)",1.0,"( 0.8, 1.1)",19.7,"(15.0, 25.7)","(42.27079011250, -83.7420214807)",400500


In [11]:
"""
filter the images to Ann Arbor
"""
data_df = data_df[data_df['TRACT'].isin(cities_df['TRACT'])]
print(data_df.shape)

CITIES_COLS = cities_df.columns.to_list()
data_df = pd.merge(data_df, cities_df, on='TRACT', how='left')
print(data_df.shape)

(208, 3881)
(208, 3944)


In [14]:
"""
a list of crude prevalences
"""
ALL_LABEL_COLS = [x for x in data_df.columns if 'label' in x]
print(f"Number of Distinct Labels: {len(ALL_LABEL_COLS)}")

MANUAL_LABELS = [x.strip() for x in open("../image_data/manual_labels.txt")]
print(f"Number of Distinct Labels in curated list: {len(MANUAL_LABELS)}")

PREVALENCE = [x for x in CITIES_COLS if 'CrudePrev' in x]
print(f"Number prevalence columns: {len(PREVALENCE)}")

REDS = [x for x in data_df.columns if 'red_eig' in x]
BLUES = [x for x in data_df.columns if 'blue_eig' in x]
GREENS = [x for x in data_df.columns if 'green_eig' in x]

Number of Distinct Labels: 326
Number of Distinct Labels in curated list: 99
Number prevalence columns: 28


In [15]:
PREVALENCE

['ACCESS2_CrudePrev',
 'ARTHRITIS_CrudePrev',
 'BINGE_CrudePrev',
 'BPHIGH_CrudePrev',
 'BPMED_CrudePrev',
 'CANCER_CrudePrev',
 'CASTHMA_CrudePrev',
 'CHD_CrudePrev',
 'CHECKUP_CrudePrev',
 'CHOLSCREEN_CrudePrev',
 'COLON_SCREEN_CrudePrev',
 'COPD_CrudePrev',
 'COREM_CrudePrev',
 'COREW_CrudePrev',
 'CSMOKING_CrudePrev',
 'DENTAL_CrudePrev',
 'DIABETES_CrudePrev',
 'HIGHCHOL_CrudePrev',
 'KIDNEY_CrudePrev',
 'LPA_CrudePrev',
 'MAMMOUSE_CrudePrev',
 'MHLTH_CrudePrev',
 'OBESITY_CrudePrev',
 'PAPTEST_CrudePrev',
 'PHLTH_CrudePrev',
 'SLEEP_CrudePrev',
 'STROKE_CrudePrev',
 'TEETHLOST_CrudePrev']

In [20]:
# """
# Let's do tttest for each label 
# """

# NAN_POLICY = 'omit'

# for prev in PREVALENCE:
#     TARGET_COL = prev
#     new_rows = []
#     for col in MANUAL_LABELS:

#         labeled = tmp[tmp[col] == 1]
#         unlabeled = tmp[tmp[col] == 0]

#         T, pval = scipy.stats.ttest_ind(labeled[TARGET_COL], 
#                             unlabeled[TARGET_COL], 
#                             nan_policy=NAN_POLICY)

#         row = {
#             'Label':col,
#             'N_labeled':len(labeled),
#             'N_unlabeled':len(unlabeled),
#             'Labeled_mean':labeled[TARGET_COL].mean(),
#             'Unabeled_mean':unlabeled[TARGET_COL].mean(),
#             'T-Obs':T,
#             'p-Value':pval,
#             'Bonferroni_Alpha': 1 - (1 - 0.05)**0.5
#         }

#         new_rows.append(row)

#     res = pd.DataFrame(new_rows)
#     res['Reject'] = np.where(res['p-Value'] <= res['Bonferroni_Alpha'], 'True', 'False' )


#     TITLE = f'Significant Differences in {TARGET_COL} by Google Label'

#     sigs = res[res['Reject'] == 'True']
#     sigs.set_index('Label', inplace=True)
#     sigs = sigs[['Labeled_mean', 'Unabeled_mean']]
#     sigs.plot(kind='barh', edgecolor='black', figsize=(8, 12), title=TITLE)
#     plt.grid()
#     plt.xlabel(f'{TARGET_COL}', fontsize=8)
#     plt.gca().invert_yaxis()
#     save_filepath = f"../figures/{TITLE}.png"
#     plt.savefig(save_filepath,  bbox_inches = 'tight')
